In [22]:
import pandas as pd

In [23]:
pop = pd.read_csv('./population_by_zip_2010.csv')
land = pd.read_csv('./2020_Gaz_zcta_national.csv')

In [24]:
# land.head()

In [40]:
# get only GEOID and ALAND, exclude other fields I'm not interested
land = land[['GEOID', 'ALAND_SQMI']]
# land.head()

In [26]:
zips = [   # zip codes wanted for San Diego
92103, 92115, 92104,
92111, 92110, 92105,
92120, 92122, 92102,
92106, 92126, 92107,
92127, 92117, 92131,
92108, 92116, 92130,
92101, 92118, 92129,
92119, 92113, 92154,
92128, 92109, 92114,
92139, 92123]

In [27]:
"""Testing masks"""
tmp = [True, False, True]
tmp_se = pd.Series(tmp)
tmp_2 = [True, False, False]
tmp_se2 = pd.Series(tmp_2)
~(tmp_se | tmp_se2)

0    False
1     True
2    False
dtype: bool

In [28]:
# pop.head()

In [29]:
# pop.describe()

In [30]:
# pop.loc[pop['minimum_age'].isnull()]
mask_min_age = pop['minimum_age'].isnull()
mask_max_age = pop['maximum_age'].isnull()
mask_max_gender = pop['gender'].isnull()
mask = (mask_min_age & mask_max_age & mask_max_gender)


In [31]:
# remove duplicate segments
pop = pop.loc[mask]
# pop

In [32]:
# verify cleaning result using total US population (~300M)
pop['population'].sum()

312444427

In [33]:
# get only population and zipcode field
pop = pop[['population', 'zipcode']]
# pop.head()

In [34]:
# join pop and land dataframe
tb = pop.set_index('zipcode').join(land.set_index('GEOID'))
# tb.head()

In [35]:
pop_dens = tb.copy()
pop_dens['pop_density'] = pop_dens['population'] / pop_dens['ALAND_SQMI']
pop_dens = pop_dens.rename({'ALAND_SQMI': 'area (sq mi)'}, axis=1)
pop_dens.head()

,population,area (sq mi),pop_density
zipcode,,,
91324,27669,4.389,6304.169515
72632,4449,99.264,44.819874
72739,473,5.143,91.969668
27305,1685,26.821,62.823907
1843,24425,3.373,7241.328194


In [36]:
pop_dens

,population,area (sq mi),pop_density
zipcode,,,
91324,27669,4.389,6304.169515
72632,4449,99.264,44.819874
72739,473,5.143,91.969668
27305,1685,26.821,62.823907
1843,24425,3.373,7241.328194
...,...,...,...
59937,12588,300.975,41.824072
75645,9778,145.050,67.411238
63855,1117,78.593,14.212462


In [37]:
pop_dens.isnull().any()

population      False
area (sq mi)    False
pop_density     False
dtype: bool

In [38]:
pop_dens.describe()

,population,area (sq mi),pop_density
count,33119.000000,33119.000000,33119.000000
mean,9433.993387,86.649685,1269.905789
std,13754.055366,253.617821,4976.384090
min,0.000000,0.002000,0.000000
25%,734.000000,9.053500,21.100766
50%,2805.000000,35.877000,81.446420
75%,12942.000000,88.560000,668.300170
max,113916.000000,13464.318000,156666.666667


In [39]:
# inspect the max density region. Turned out to be a mini zone in DC
pop_dens.loc[pop_dens['pop_density'] == max(pop_dens['pop_density'])]

,population,area (sq mi),pop_density
zipcode,,,
20052,470,0.003,156666.666667


In [41]:
'''Export to .csv'''
pop_dens.to_csv('./population_density_by_zipcode.csv')